# **Пример с хаггинфэйс на претренированном датасете**

In [1]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

sequence_to_classify = "Тейлор Свифт поп и кантри певица"
candidate_labels = ["politics", "economy", "entertainment", "environment"]

output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'Тейлор Свифт поп и кантри певица', 'labels': ['entertainment', 'environment', 'economy', 'politics'], 'scores': [0.8593410849571228, 0.05369015410542488, 0.049566809087991714, 0.03740197792649269]}


In [2]:
pip install transformers datasets evaluate accelerate

In [4]:
!pip install optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# **Датасет lenta.ru 2019-2023**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
file_path = '/content/drive/MyDrive/lenta_ru_news_2019_2023.csv'
df = pd.read_csv(file_path, sep=',', header=0)

In [7]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2019/12/15/prsm/,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия,Общество,2019-12-15
1,https://lenta.ru/news/2019/12/15/fb/,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия,Политика,2019-12-15
2,https://lenta.ru/news/2019/12/15/kino/,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура,Фотография,2019-12-15
3,https://lenta.ru/news/2019/12/15/alba/,Полицейские застрелили порезавшего мать буйног...,В Москве полицейские застрелили мужчину при по...,Силовые структуры,Криминал,2019-12-15
4,https://lenta.ru/news/2019/12/15/anons/,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир,Политика,2019-12-15


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496257 entries, 0 to 496256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     496257 non-null  object
 1   title   496255 non-null  object
 2   text    496257 non-null  object
 3   topic   496173 non-null  object
 4   tags    491110 non-null  object
 5   date    496257 non-null  object
dtypes: object(6)
memory usage: 22.7+ MB


In [9]:
df = df.dropna()
df['topic'] = df['topic'].astype('category')

<ipython-input-9-50d310c8e485>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic'] = df['topic'].astype('category')


In [10]:
df.value_counts('tags').head(30)

,count
tags,
Политика,97275
Общество,55367
Украина,33421
Происшествия,21214
Следствие и суд,18994
Госэкономика,17442
Дом,11794
Футбол,9567
Рынки,9126


In [11]:
# Define the list of tags you want to keep
tags_to_keep = ['Политика', 'Госэкономика', 'Происшествия', 'Бизнес', 'Кино', 'Музыка', 'Внешний вид', 'Город', 'Стиль', 'Рынки', 'Футбол', 'Зимние виды']

# Filter the DataFrame to only include rows with tags in the list
filtered_df = df[df['tags'].isin(tags_to_keep)]

Здесь были вручную выбраны темы, которые точно политические (tag Политика), и разнообразные точно не политические (tag Госэкономика, Происшествия, Бизнес, Кино, Музыка, Внешний вид, Город, Стиль, Рынки, Футбол, Зимние виды). Оба эти класса были выбраны таким образом, чтобы результат политика/не политика был примерно 50/50.

In [12]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194762 entries, 1 to 496256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   url     194762 non-null  object  
 1   title   194762 non-null  object  
 2   text    194762 non-null  object  
 3   topic   194762 non-null  category
 4   tags    194762 non-null  object  
 5   date    194762 non-null  object  
dtypes: category(1), object(5)
memory usage: 9.1+ MB


In [13]:
filtered_df['is_politics'] = filtered_df['tags'].apply(lambda x: 1 if x == 'Политика' else 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491110 entries, 0 to 496256
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   url     491110 non-null  object  
 1   title   491110 non-null  object  
 2   text    491110 non-null  object  
 3   topic   491110 non-null  category
 4   tags    491110 non-null  object  
 5   date    491110 non-null  object  
dtypes: category(1), object(5)
memory usage: 23.0+ MB


<ipython-input-13-ddc9f5b12375>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['is_politics'] = filtered_df['tags'].apply(lambda x: 1 if x == 'Политика' else 0)


In [14]:
# Случайным образом отбираем 50000 строк
sampled_df = filtered_df.sample(n=50000, random_state=42)

In [15]:
# Делим данные на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(sampled_df['text'], sampled_df['is_politics'], test_size=0.25, random_state=42,stratify=sampled_df['is_politics'])

# Подход Логистическая регрессия

In [16]:
# Векторизация с помощью TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [17]:
# Обучаем модель логистической регрессии на трейне
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

LogisticRegression()

In [18]:
# Прогнозируем целевую переменную на тестовой выборке
y_pred = classifier.predict(X_test_tfidf)

In [19]:
# Оцениваем качество модели
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      6260
           1       0.94      0.94      0.94      6240

    accuracy                           0.94     12500
   macro avg       0.94      0.94      0.94     12500
weighted avg       0.94      0.94      0.94     12500



# Попытаемся улучшить токенизацию тектстов

Используя BPE и word2vec

In [20]:
# Импортируем необходимый класс AutoTokenizer из библиотеки transformers
from transformers import AutoTokenizer

# Создаём экземпляр токенизатора, загружая предварительно обученный токенизатор соответствующей модели
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [21]:
print(sampled_df.columns)

Index(['url', 'title', 'text', 'topic', 'tags', 'date', 'is_politics'], dtype='object')


In [22]:
def tokenize_text(text):
    # Tokenize the text and convert to token IDs
    return tokenizer.encode(text, add_special_tokens=True)

# Apply the tokenizer to the 'text' column
sampled_df['tokens'] = sampled_df['text'].apply(tokenize_text)

In [23]:
# Инициализируем токенизатор модели
def tokenize_text(text):
    return tokenizer.tokenize(text)

# Применяем функцию к столбцу 'text'
sampled_df['tokens'] = sampled_df['text'].apply(tokenize_text)

In [24]:
sampled_df.head()

,url,title,text,topic,tags,date,is_politics,tokens
340444,https://lenta.ru/news/2022/11/10/sobakaa/,Пережившая нападение акулы женщина рассказала ...,50-летняя жительница США Лин Ютронич пережила ...,Из жизни,Происшествия,2022-11-10,0,"[50, -, летняя, жительница, США, Лин, Ют, ##ро..."
481461,https://lenta.ru/news/2023/11/29/korabl-progre...,Российским чиновникам предложили курировать се...,Российским чиновникам предложили курировать се...,Россия,Политика,2023-11-29,1,"[Российским, чиновникам, предложили, курироват..."
224400,https://lenta.ru/news/2022/02/10/003879504/,Евросоюз ответил на письмо Лаврова по безопасн...,Евросоюз (ЕС) передал официальный коллективный...,Мир,Политика,2022-02-10,1,"[Евросоюз, (, ЕС, ), передал, официальный, кол..."
90318,https://lenta.ru/news/2021/01/02/pasynok/,Байден заявил о желании сотрудничать с Констан...,Избранный президент США Джо Байден в ответном ...,Мир,Политика,2021-01-02,1,"[Избранный, президент, США, Джо, Байден, в, от..."
89047,https://lenta.ru/news/2020/12/27/levye/,Трамп призвал республиканцев бороться за его п...,Президент США Дональд Трамп призвал Республика...,Мир,Политика,2020-12-27,1,"[Президент, США, Дональд, Трамп, призвал, Респ..."


In [25]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Загружаем список стопслов
russian_stopwords = set(stopwords.words('russian'))

# Инициализируем функцию, которая удаляет стопслова из токенизированного текста
def remove_stopwords(tokens):
    return [token for token in tokens if token not in russian_stopwords]

# Удаляем стопслова из токенизированного текста
sampled_df['tokens_no_stopwords'] = sampled_df['tokens'].apply(remove_stopwords)

sampled_df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,title,text,topic,tags,date,is_politics,tokens,tokens_no_stopwords
340444,https://lenta.ru/news/2022/11/10/sobakaa/,Пережившая нападение акулы женщина рассказала ...,50-летняя жительница США Лин Ютронич пережила ...,Из жизни,Происшествия,2022-11-10,0,"[50, -, летняя, жительница, США, Лин, Ют, ##ро...","[50, -, летняя, жительница, США, Лин, Ют, ##ро..."
481461,https://lenta.ru/news/2023/11/29/korabl-progre...,Российским чиновникам предложили курировать се...,Российским чиновникам предложили курировать се...,Россия,Политика,2023-11-29,1,"[Российским, чиновникам, предложили, курироват...","[Российским, чиновникам, предложили, курироват..."
224400,https://lenta.ru/news/2022/02/10/003879504/,Евросоюз ответил на письмо Лаврова по безопасн...,Евросоюз (ЕС) передал официальный коллективный...,Мир,Политика,2022-02-10,1,"[Евросоюз, (, ЕС, ), передал, официальный, кол...","[Евросоюз, (, ЕС, ), передал, официальный, кол..."
90318,https://lenta.ru/news/2021/01/02/pasynok/,Байден заявил о желании сотрудничать с Констан...,Избранный президент США Джо Байден в ответном ...,Мир,Политика,2021-01-02,1,"[Избранный, президент, США, Джо, Байден, в, от...","[Избранный, президент, США, Джо, Байден, ответ..."
89047,https://lenta.ru/news/2020/12/27/levye/,Трамп призвал республиканцев бороться за его п...,Президент США Дональд Трамп призвал Республика...,Мир,Политика,2020-12-27,1,"[Президент, США, Дональд, Трамп, призвал, Респ...","[Президент, США, Дональд, Трамп, призвал, Респ..."


In [26]:
# Разбиваем данные на трейн и тест еще раз
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(sampled_df['tokens_no_stopwords'], sampled_df['is_politics'], test_size=0.25, random_state=42, stratify=sampled_df['is_politics'])


In [27]:
import gensim

# Обучаем Word2Vec модель
word2vec_model = gensim.models.Word2Vec(sentences=X_train_2, vector_size=100, window=5, min_count=1, workers=4)

# Convert the tokenized text data into Word2Vec embeddings
def tokens_to_embedding(tokens, model):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

In [28]:
# Apply the tokens_to_embedding function to the train and test sets
X_train_embeddings = np.array([tokens_to_embedding(tokens, word2vec_model) for tokens in X_train_2])
X_test_embeddings = np.array([tokens_to_embedding(tokens, word2vec_model) for tokens in X_test_2])


In [29]:
# Train the model
classifier = LogisticRegression()
classifier.fit(X_train_embeddings, y_train_2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
# Оцениваем модель
y_pred_2 = classifier.predict(X_test_embeddings)

In [31]:
# Оцениваем качество модели
print(classification_report(y_test_2, y_pred_2))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      6260
           1       0.92      0.92      0.92      6240

    accuracy                           0.92     12500
   macro avg       0.92      0.92      0.92     12500
weighted avg       0.92      0.92      0.92     12500



# Подход Решающие деревья

In [63]:
# Objective function for Optuna
def objective(trial):
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    splitter = trial.suggest_categorical('splitter', ['best', 'random'])
    max_depth = trial.suggest_int('max_depth', 1, 40)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    dt_classifier = DecisionTreeClassifier(
        criterion=criterion,
        splitter=splitter,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    dt_classifier.fit(X_train_tfidf, y_train)
    y_pred = dt_classifier.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


In [64]:
# Create a study and optimize it
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-08-01 15:37:40,748] A new study created in memory with name: no-name-9e587f99-8355-4d57-afbd-4e8520ecf241
[I 2024-08-01 15:37:45,351] Trial 0 finished with value: 0.8044 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 9, 'min_samples_split': 15, 'min_samples_leaf': 19}. Best is trial 0 with value: 0.8044.
[I 2024-08-01 15:37:50,188] Trial 1 finished with value: 0.8192 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 9, 'min_samples_split': 16, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.8192.
[I 2024-08-01 15:37:59,915] Trial 2 finished with value: 0.8432 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 14, 'min_samples_split': 19, 'min_samples_leaf': 6}. Best is trial 2 with value: 0.8432.
[I 2024-08-01 15:38:25,162] Trial 3 finished with value: 0.85288 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 33, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is tri

In [65]:
# Best parameters
print("Best parameters: ", study.best_params)

Best parameters:  {'criterion': 'gini', 'splitter': 'random', 'max_depth': 39, 'min_samples_split': 9, 'min_samples_leaf': 1}


In [66]:
# Train and evaluate Decision Tree with best parameters
best_params = study.best_params
optimized_dt_classifier = DecisionTreeClassifier(
    criterion=best_params['criterion'],
    splitter=best_params['splitter'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
optimized_dt_classifier.fit(X_train_tfidf, y_train)
y_pred_optimized_dt = optimized_dt_classifier.predict(X_test_tfidf)
print("Optimized Decision Tree Classifier:\n", classification_report(y_test, y_pred_optimized_dt))

Optimized Decision Tree Classifier:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87      6260
           1       0.87      0.87      0.87      6240

    accuracy                           0.87     12500
   macro avg       0.87      0.87      0.87     12500
weighted avg       0.87      0.87      0.87     12500



# Подход Случайный лес

In [1]:
# Objective function for Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 180)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 14)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 14)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    rf_classifier = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    rf_classifier.fit(X_train_tfidf, y_train)
    y_pred = rf_classifier.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
# Create a study and optimize it
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-08-01 17:00:09,730] A new study created in memory with name: no-name-3f07baf0-0051-4958-aee2-dcf89003c6ed
[I 2024-08-01 17:00:15,326] Trial 0 finished with value: 0.89032 and parameters: {'n_estimators': 179, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.89032.


In [ ]:
# Best parameters
print("Best parameters: ", study.best_params)

In [ ]:
# Train and evaluate Random Forest with best parameters
best_params = study.best_params
optimized_rf_classifier = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    random_state=42
)
optimized_rf_classifier.fit(X_train_tfidf, y_train)
y_pred_optimized_rf = optimized_rf_classifier.predict(X_test_tfidf)
print("Optimized Random Forest Classifier:\n", classification_report(y_test, y_pred_optimized_rf))